In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

<cell_type>markdown</cell_type># DermaScan AI: Vision Encoders for Automated Dermatology Screening

**Implementation Notebook -- Vizuara Case Study**

In this notebook, we implement the full pipeline for DermaScan AI's automated dermatology screening system using real dermatoscopic images from DermaMNIST:
1. Data loading with class-weighted sampling
2. CNN (ResNet-18) and ViT (ViT-B/16) encoders
3. Custom loss function (weighted CE + focal loss)
4. Training, evaluation, and error analysis
5. Deployment optimization

**Dataset:** DermaMNIST contains 10,015 real dermatoscopic images across 7 skin lesion types, sourced from the HAM10000 dataset used in clinical research. This is the real version of the task DermaScan AI needs to solve.

**Runtime:** Google Colab (T4 GPU required)
**Estimated time:** 90-120 minutes

In [ ]:
# Setup and dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from torch.utils.data import Dataset
import time

!pip install -q medmnist
import medmnist
from medmnist import DermaMNIST

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# DermaMNIST: real dermatoscopic images, 7 classes
# Classes: 0=actinic_keratosis, 1=basal_cell_carcinoma, 2=benign_keratosis,
#          3=dermatofibroma, 4=melanoma, 5=melanocytic_nevus, 6=vascular_lesion
NUM_CLASSES = 7
CLASS_NAMES = ['actinic_keratosis', 'basal_cell_carcinoma', 'benign_keratosis',
               'dermatofibroma', 'melanoma', 'melanocytic_nevus', 'vascular_lesion']
MELANOMA_IDX = 4  # Melanoma is class 4 in DermaMNIST

<cell_type>markdown</cell_type>## Section 3.1: Data Loading and Preprocessing

**TODO 1:** Implement the data loading pipeline with class-weighted sampling to handle the severe class imbalance in DermaMNIST (melanocytic nevi dominate the dataset).

In [ ]:
class IntLabelDataset(Dataset):
    """Wraps a MedMNIST dataset to return integer labels and support .targets attribute."""
    def __init__(self, dataset):
        self.dataset = dataset
        # Pre-extract all labels for WeightedRandomSampler
        self.targets = [int(dataset[i][1].item()) for i in range(len(dataset))]
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, int(label.item())

def load_dermascan_dataset(img_size=224):
    """
    Load and preprocess the DermaMNIST dermatoscopic image dataset.

    Returns:
        train_loader, val_loader, test_loader, class_weights
    """
    # Training transforms with medical imaging augmentation
    transform_train = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.1),
    ])

    transform_test = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    # Load real DermaMNIST data
    train_raw = DermaMNIST(split='train', download=True, transform=transform_train)
    test_raw = DermaMNIST(split='test', download=True, transform=transform_test)

    trainset = IntLabelDataset(train_raw)
    testset = IntLabelDataset(test_raw)

    # Compute class weights
    class_counts = np.bincount(trainset.targets, minlength=NUM_CLASSES)
    class_weights = len(trainset.targets) / (NUM_CLASSES * class_counts + 1e-6)
    class_weights = torch.FloatTensor(class_weights)
    print(f'Class distribution: {dict(zip(CLASS_NAMES, class_counts))}')
    print(f'Class weights: {class_weights.numpy().round(2)}')

    # Weighted sampler for balanced batches
    sample_weights = class_weights[trainset.targets]
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=sample_weights, num_samples=len(trainset), replacement=True)

    train_loader = torch.utils.data.DataLoader(
        trainset, batch_size=64, sampler=sampler, num_workers=2)

    # Split test into val/test
    val_size = len(testset) // 2
    val_set, test_set = torch.utils.data.random_split(
        testset, [val_size, len(testset) - val_size])
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, num_workers=2)

    return train_loader, val_loader, test_loader, class_weights

train_loader, val_loader, test_loader, class_weights = load_dermascan_dataset(img_size=32)

<cell_type>markdown</cell_type>## Section 3.2: Exploratory Data Analysis

**TODO 2:** Visualize the DermaMNIST dataset to understand class distribution and image characteristics.

In [ ]:
def run_eda(train_loader, class_names):
    """
    TODO: Implement exploratory data analysis.
    
    1. Plot class distribution bar chart
    2. Display sample images from each class
    3. Compute mean pixel intensity per class
    """
    # Collect a batch of images
    images, labels = next(iter(train_loader))
    
    # Plot class distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    axes[0].bar(range(len(class_names)), [counts[counts==i].sum() if i in unique else 0 
                for i in range(len(class_names))], color='steelblue')
    axes[0].set_xticks(range(len(class_names)))
    axes[0].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[0].set_ylabel('Count (batch)')
    axes[0].set_title('Class Distribution in Batch')
    
    # Show sample images
    fig2, axes2 = plt.subplots(2, 7, figsize=(16, 5))
    fig2.suptitle('Sample Images per Class', fontsize=14)
    for cls in range(min(7, len(class_names))):
        mask = labels == cls
        if mask.sum() > 0:
            idx = mask.nonzero()[0][0]
            img = images[idx].permute(1, 2, 0).numpy()
            img = (img - img.min()) / (img.max() - img.min())
            axes2[0, cls].imshow(img)
            axes2[0, cls].set_title(class_names[cls], fontsize=8)
            axes2[0, cls].axis('off')
            if mask.sum() > 1:
                idx2 = mask.nonzero()[0][1]
                img2 = images[idx2].permute(1, 2, 0).numpy()
                img2 = (img2 - img2.min()) / (img2.max() - img2.min())
                axes2[1, cls].imshow(img2)
            axes2[1, cls].axis('off')
    plt.tight_layout()
    plt.show()

run_eda(train_loader, CLASS_NAMES)

## Section 3.3: Baseline Model (ResNet-50)

**TODO 3:** Create the ResNet-50 baseline with pretrained ImageNet weights.

In [ ]:
def create_baseline_model(num_classes=7, pretrained=True, img_size=32):
    """
    TODO: Create a ResNet-50 baseline model.
    
    Steps:
    1. Load ResNet-50 with pretrained weights
    2. Replace final fc layer for num_classes outputs
    3. If img_size < 224, modify first conv to handle smaller inputs
    """
    # For small images (CIFAR), use ResNet-18 instead
    if img_size <= 64:
        model = torchvision.models.resnet18(weights='IMAGENET1K_V1' if pretrained else None)
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()
        model.fc = nn.Linear(512, num_classes)
    else:
        model = torchvision.models.resnet50(weights='IMAGENET1K_V2' if pretrained else None)
        model.fc = nn.Linear(2048, num_classes)
    return model

baseline = create_baseline_model(img_size=32).to(device)
print(f'Baseline params: {sum(p.numel() for p in baseline.parameters()):,}')

## Section 3.4: Vision Transformer Model

**TODO 4:** Create the ViT model and implement the custom loss function.

In [ ]:
class SimpleViTForMedical(nn.Module):
    """Vision Transformer adapted for small medical images."""
    def __init__(self, img_size=32, patch_size=4, embed_dim=192,
                 num_heads=6, num_layers=6, num_classes=7):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embed = nn.Conv2d(3, embed_dim, patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, embed_dim) * 0.02)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim * 4,
            activation='gelu', batch_first=True, norm_first=True, dropout=0.1)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = torch.cat([self.cls_token.expand(B, -1, -1), x], dim=1) + self.pos_embed
        x = self.norm(self.transformer(x))
        return self.head(x[:, 0])

vit_model = SimpleViTForMedical(img_size=32).to(device)
print(f'ViT params: {sum(p.numel() for p in vit_model.parameters()):,}')

In [ ]:
class DermaScanLoss(nn.Module):
    """
    TODO 5: Implement the combined loss function.
    
    Loss = Weighted_CE + lambda_1 * FocalLoss(melanoma) + lambda_2 * L2_reg
    
    Steps:
    1. Compute weighted cross-entropy
    2. Compute focal loss for melanoma class:
       FL(p_t) = -(1-p_t)^gamma * log(p_t)
    3. Return weighted combination
    """
    def __init__(self, class_weights, melanoma_idx=0, focal_gamma=2.0, focal_weight=0.3):
        super().__init__()
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights)
        self.melanoma_idx = melanoma_idx
        self.focal_gamma = focal_gamma
        self.focal_weight = focal_weight

    def forward(self, logits, targets):
        # Standard weighted cross-entropy
        ce = self.ce_loss(logits, targets)

        # Focal loss for melanoma
        probs = F.softmax(logits, dim=-1)
        mel_prob = probs[:, self.melanoma_idx]
        mel_mask = (targets == self.melanoma_idx).float()

        # For melanoma samples: FL = -(1-p_mel)^gamma * log(p_mel)
        # For non-melanoma: FL = -(p_mel)^gamma * log(1-p_mel)
        p_t = mel_prob * mel_mask + (1 - mel_prob) * (1 - mel_mask)
        focal = -((1 - p_t) ** self.focal_gamma) * torch.log(p_t + 1e-8)
        focal_loss = focal.mean()

        return ce + self.focal_weight * focal_loss

loss_fn = DermaScanLoss(class_weights.to(device))
print('Loss function created with weighted CE + melanoma focal loss')

## Section 3.5: Training Pipeline

In [ ]:
def train_and_evaluate(model, train_loader, val_loader, loss_fn,
                       num_epochs=15, lr=1e-3, model_name='model'):
    """Train model and track melanoma-specific metrics."""
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    history = {'train_loss': [], 'val_acc': [], 'mel_sensitivity': [], 'mel_auroc': []}
    best_mel_sens = 0
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        
        # Validation
        model.eval()
        all_preds, all_labels, all_probs = [], [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                probs = F.softmax(outputs, dim=-1)
                _, preds = outputs.max(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        val_acc = (all_preds == all_labels).mean() * 100
        
        # Melanoma metrics
        mel_true = (all_labels == MELANOMA_IDX).astype(int)
        mel_pred = (all_preds == MELANOMA_IDX).astype(int)
        mel_tp = ((mel_pred == 1) & (mel_true == 1)).sum()
        mel_fn = ((mel_pred == 0) & (mel_true == 1)).sum()
        mel_sensitivity = mel_tp / (mel_tp + mel_fn + 1e-8) * 100
        
        try:
            mel_auroc = roc_auc_score(mel_true, all_probs[:, MELANOMA_IDX])
        except:
            mel_auroc = 0.5
        
        history['train_loss'].append(running_loss / len(train_loader))
        history['val_acc'].append(val_acc)
        history['mel_sensitivity'].append(mel_sensitivity)
        history['mel_auroc'].append(mel_auroc)
        
        if mel_sensitivity > best_mel_sens:
            best_mel_sens = mel_sensitivity
            best_state = model.state_dict().copy()
        
        if (epoch + 1) % 5 == 0:
            print(f'  Epoch {epoch+1}/{num_epochs}: Loss={running_loss/len(train_loader):.3f}, '
                  f'Acc={val_acc:.1f}%, Mel Sens={mel_sensitivity:.1f}%, AUROC={mel_auroc:.3f}')
    
    return history, best_state

In [ ]:
# Train both models
print('Training ResNet-18 Baseline...')
baseline = create_baseline_model(img_size=32).to(device)
loss_fn = DermaScanLoss(class_weights.to(device))
resnet_history, resnet_best = train_and_evaluate(
    baseline, train_loader, val_loader, loss_fn, num_epochs=15, model_name='resnet')

print('\nTraining Vision Transformer...')
vit_model = SimpleViTForMedical(img_size=32).to(device)
loss_fn = DermaScanLoss(class_weights.to(device))
vit_history, vit_best = train_and_evaluate(
    vit_model, train_loader, val_loader, loss_fn, num_epochs=15, lr=3e-4, model_name='vit')

## Section 3.6: Evaluation

**TODO 6:** Compare both models on test data and compute all required metrics.

In [ ]:
def full_evaluation(model, test_loader, model_name='Model'):
    """Comprehensive evaluation with confusion matrix and ROC curves."""
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            probs = F.softmax(model(images), dim=-1)
            _, preds = probs.max(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    # Overall metrics
    accuracy = (all_preds == all_labels).mean() * 100
    
    # Melanoma metrics
    mel_true = (all_labels == MELANOMA_IDX).astype(int)
    mel_pred = (all_preds == MELANOMA_IDX).astype(int)
    mel_tp = ((mel_pred == 1) & (mel_true == 1)).sum()
    mel_fp = ((mel_pred == 1) & (mel_true == 0)).sum()
    mel_fn = ((mel_pred == 0) & (mel_true == 1)).sum()
    mel_tn = ((mel_pred == 0) & (mel_true == 0)).sum()
    
    mel_sens = mel_tp / (mel_tp + mel_fn + 1e-8) * 100
    mel_spec = mel_tn / (mel_tn + mel_fp + 1e-8) * 100
    mel_auroc = roc_auc_score(mel_true, all_probs[:, MELANOMA_IDX])
    
    print(f'\n{model_name} Results:')
    print(f'  Overall Accuracy: {accuracy:.1f}%')
    print(f'  Melanoma Sensitivity: {mel_sens:.1f}% (target: >90%)')
    print(f'  Melanoma Specificity: {mel_spec:.1f}% (target: >85%)')
    print(f'  Melanoma AUROC: {mel_auroc:.3f}')
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].imshow(cm, cmap='Blues')
    axes[0].set_xticks(range(NUM_CLASSES))
    axes[0].set_yticks(range(NUM_CLASSES))
    axes[0].set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=7)
    axes[0].set_yticklabels(CLASS_NAMES, fontsize=7)
    axes[0].set_title(f'{model_name} Confusion Matrix')
    for i in range(NUM_CLASSES):
        for j in range(NUM_CLASSES):
            axes[0].text(j, i, str(cm[i, j]), ha='center', va='center', fontsize=8)
    
    # ROC curve for melanoma
    fpr, tpr, _ = roc_curve(mel_true, all_probs[:, MELANOMA_IDX])
    axes[1].plot(fpr, tpr, 'b-', linewidth=2, label=f'AUROC={mel_auroc:.3f}')
    axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.3)
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title(f'{model_name} Melanoma ROC Curve')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'accuracy': accuracy, 'mel_sensitivity': mel_sens,
            'mel_specificity': mel_spec, 'mel_auroc': mel_auroc}

# Evaluate both
baseline.load_state_dict(resnet_best)
resnet_metrics = full_evaluation(baseline, test_loader, 'ResNet-18')

vit_model.load_state_dict(vit_best)
vit_metrics = full_evaluation(vit_model, test_loader, 'ViT')

## Section 3.7: Error Analysis

**TODO 7:** Analyze the model's failure cases, particularly false negative melanomas.

In [ ]:
def error_analysis(model, test_loader, model_name='Model'):
    """
    TODO: Analyze failure cases.
    
    1. Find false negative melanomas (predicted non-melanoma, true melanoma)
    2. Visualize the top 5 most confident false negatives
    3. Compute error rate per class
    """
    model.eval()
    false_negatives = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            probs = F.softmax(model(images), dim=-1)
            preds = probs.argmax(1)
            
            # Find false negatives for melanoma
            mel_mask = labels == MELANOMA_IDX
            wrong_mask = preds != MELANOMA_IDX
            fn_mask = mel_mask & wrong_mask
            
            for i in fn_mask.nonzero():
                idx = i.item()
                false_negatives.append({
                    'image': images[idx].cpu(),
                    'mel_prob': probs[idx, MELANOMA_IDX].item(),
                    'predicted': CLASS_NAMES[preds[idx].item()]
                })
    
    if false_negatives:
        false_negatives.sort(key=lambda x: x['mel_prob'])
        print(f'\n{model_name}: {len(false_negatives)} false negative melanomas')
        
        n_show = min(5, len(false_negatives))
        fig, axes = plt.subplots(1, n_show, figsize=(3*n_show, 3))
        if n_show == 1: axes = [axes]
        for i in range(n_show):
            img = false_negatives[i]['image'].permute(1, 2, 0).numpy()
            img = (img - img.min()) / (img.max() - img.min())
            axes[i].imshow(img)
            axes[i].set_title(f"Pred: {false_negatives[i]['predicted']}\n"
                             f"Mel prob: {false_negatives[i]['mel_prob']:.3f}", fontsize=9)
            axes[i].axis('off')
        plt.suptitle(f'{model_name}: Missed Melanomas', fontsize=13)
        plt.tight_layout()
        plt.show()
    else:
        print(f'{model_name}: No false negative melanomas!')

error_analysis(baseline, test_loader, 'ResNet-18')
error_analysis(vit_model, test_loader, 'ViT')

## Section 3.8: Deployment Optimization

In [ ]:
def benchmark_inference(model, img_size=32, batch_size=1, num_runs=100):
    """Benchmark inference speed."""
    model.eval()
    dummy = torch.randn(batch_size, 3, img_size, img_size).to(device)
    
    # Warmup
    for _ in range(10):
        with torch.no_grad():
            _ = model(dummy)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    start = time.time()
    for _ in range(num_runs):
        with torch.no_grad():
            _ = model(dummy)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    elapsed = time.time() - start
    
    latency_ms = (elapsed / num_runs) * 1000
    throughput = num_runs / elapsed
    print(f'  Latency: {latency_ms:.1f} ms/image')
    print(f'  Throughput: {throughput:.0f} images/sec')
    return latency_ms

print('ResNet-18 Inference:')
resnet_latency = benchmark_inference(baseline)

print('\nViT Inference:')
vit_latency = benchmark_inference(vit_model)

print(f'\nViT is {vit_latency/resnet_latency:.1f}x slower than ResNet-18')

## Section 3.9: Ethical Considerations and Summary

In [ ]:
# Final comparison summary
print('=' * 60)
print('DermaScan AI: Model Comparison Summary')
print('=' * 60)
print(f'{"Metric":<25} {"ResNet-18":<15} {"ViT":<15} {"Target":<15}')
print('-' * 60)
print(f'{"Overall Accuracy":<25} {resnet_metrics["accuracy"]:.1f}%{"":<10} {vit_metrics["accuracy"]:.1f}%{"":<10} >84%')
print(f'{"Mel. Sensitivity":<25} {resnet_metrics["mel_sensitivity"]:.1f}%{"":<10} {vit_metrics["mel_sensitivity"]:.1f}%{"":<10} >90%')
print(f'{"Mel. Specificity":<25} {resnet_metrics["mel_specificity"]:.1f}%{"":<10} {vit_metrics["mel_specificity"]:.1f}%{"":<10} >85%')
print(f'{"Mel. AUROC":<25} {resnet_metrics["mel_auroc"]:.3f}{"":<12} {vit_metrics["mel_auroc"]:.3f}{"":<12} >0.92')
print(f'{"Inference (ms)":<25} {resnet_latency:.1f}{"":<12} {vit_latency:.1f}{"":<12} <3000')
print(f'{"Parameters":<25} {sum(p.numel() for p in baseline.parameters()):,}{"":<5} {sum(p.numel() for p in vit_model.parameters()):,}')
print('\nDataset: DermaMNIST (real dermatoscopic images from HAM10000)')
print('Ethical Note: This model must be audited for performance across')
print('all Fitzpatrick skin types (I-VI) before clinical deployment.')
print('Dermoscopy AI has documented biases against darker skin tones.')